# Use biopython to make valid CLUSTAL formatted MSAs, check sequence of manually edited alignment, and add consensus line

This is meant to represent a typical workflow where a combination of these steps would be used.

------

##  Using biopython to make valid CLUSTAL formatted MSAs

Biopython seems to not require strict adherence to line lengths for the sequence blocks. For exampe, the first line can have 83 residues for each sequence while the other lines can have the tpyical fifty or sixty residues. (CLUSTAL produced by Biopython seem to be fifty per line while those produced by EMBL-EBI are sixty.) Since it can read in uneven CLUSTAL style alignments that can arise when one tries to manually edit a multiple sequence alignment (MSA), biopython is suited to converting the hand-edited sequence to the more standardized CLUSTAL format. It is probably always best to perform this step ASAP if you have sequence alignments with uneven blocks. This way the 'standardized' CLUSTAL format will be utilized by downstream steps. Not all computational tools will be written to be as lenient as biopython is about the standard format. I personally have written scripts that rely on the first set of sequence blocks to establish the number of columns. Hence, having uneven width for the sequence blocks would cause errors.


Here a hand-edited multiple sequence alignment will be converted to more standard form.  
(The process illustrated here is very reminiscent of the section entitled 'File Format Conversion' at the wiki page for ['The module for multiple sequence alignments, AlignIO'](https://biopython.org/wiki/AlignIO).)

In [1]:
# Get an alignment file
!curl -o alignment.clw https://gist.githubusercontent.com/fomightez/f46b0624f1d8e3abb6ff908fc447e63b/raw/a3ec6fb9d5c3f558a4b8666ce1cbc20c356fe4de/unevenStv1p_Vph1p_muscle_alignment.clw

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3632  100  3632    0     0  22559      0 --:--:-- --:--:-- --:--:-- 22700


In [3]:
# Display original alignment to confirm starting state. NOTE UNEVEN LENGTH OF SEQUENCES ON LINES.
!head -12 alignment.clw

CLUSTAL multiple sequence alignment by MUSCLE (3.8)


STV1            -MNQEEAIFRSADMTYVQLYIPLEVIREVTFLLGKMSVFMVMDLNKDLTAFQRGYVNQLRRFDEVER
VPH1            MAEKEEAIFRSAEMALVQFYIPQEISRDSAYTLGQLGLVQFRDLNSKVRAFQRTFVNEIRRLDNVER
                  ::********:*: **:*** *: *: :: **::.:. . ***..: **** :**::**:*:***

STV1            MVGFLNEVVEKHAAETW-----KYILHIDDEGNDIAQPDMADLINTMEPLSLE
VPH1            QYRYFYSLLKKHDIKLYEGDTDKYL----DGSGELYVPPSGSVI---------
                   :: .:::**  : :     **:    * ..::  *  ..:*         

STV1            NVNDMVKEITDCESRARQLDESLDSLRSKLNDLLEQRQVIFECSKFIEVNPGIAGRATNP


In [4]:
# Read in original alignment
from Bio import SeqIO
from Bio import AlignIO
orig_alignment = AlignIO.read('alignment.clw', "clustal")

In [5]:
# Save original alignment as a new file
AlignIO.write(orig_alignment, 'standardized_alignment.clw', "clustal"); # based on # https://biopython.org/wiki/AlignIO;

In [7]:
#check produced file
!head -12 standardized_alignment.clw

CLUSTAL X (3.8) multiple sequence alignment


STV1                                -MNQEEAIFRSADMTYVQLYIPLEVIREVTFLLGKMSVFMVMDLNKDLTA
VPH1                                MAEKEEAIFRSAEMALVQFYIPQEISRDSAYTLGQLGLVQFRDLNSKVRA
                                      ::********:*: **:*** *: *: :: **::.:. . ***..: *

STV1                                FQRGYVNQLRRFDEVERMVGFLNEVVEKHAAETW-----KYILHIDDEGN
VPH1                                FQRTFVNEIRRLDNVERQYRYFYSLLKKHDIKLYEGDTDKYL----DGSG
                                    *** :**::**:*:***   :: .:::**  : :     **:    * ..

STV1                                DIAQPDMADLINTMEPLSLENVNDMVKEITDCESRARQLDESLDSLRSKL


-----

(Aside:   
When I was comparing the results direct from [EMBL-EBI's MUSCLE](https://www.ebi.ac.uk/Tools/msa/muscle/) to what my script to add the consensus symbols produces I noticed some discrepancies.

The third line shows that the consensus symbols for MUSCLE alignment are differently defined than other place I have seen:

    STV1            NVNDMVKEITDCESRARQLDESLDSLRSKLNDLLEQRQVIFECSKFIEVNPGIAGRATNP
    VPH1            --DDYVRNASYLEERLIQMEDATDQIEVQKNDLEQYRFILQSGDEFF-----LKGDNTDS
                      :* *.: :  *.*  *:::: *.:  : *** : * :: . .:*:     : *  *:.
                          ^                   ^            

I put an upward arrow head (super-script character) pointing out the two that
don't match what I have seen elsewhere for conserved residues.

- Why is K and R substituion not strongly similar?  Should be according to [here](https://www.ebi.ac.uk/seqdb/confluence/display/JDSAT/Bioinformatics+Tools+FAQ#BioinformaticsToolsFAQ-WhatdoconsensussymbolsrepresentinaMultipleSequenceAlignment?) and [here](https://www.genome.jp/tools/clustalw/clustalw_readme.html).
- Why is E and R substituion not weakly similar? Should be according to [here](https://www.ebi.ac.uk/seqdb/confluence/display/JDSAT/Bioinformatics+Tools+FAQ#BioinformaticsToolsFAQ-WhatdoconsensussymbolsrepresentinaMultipleSequenceAlignment?) and [here](https://www.genome.jp/tools/clustalw/clustalw_readme.html).

My script `calculate_cons_for_clustal_protein.py`, demonstrated in the last section of this notebook, annotates these correctly. I only noticed when I was trying to test `calculate_cons_for_clustal_protein.py` and noticed I wasn't producing things matching perfect with the symbols that MUSCLE adds. )

----

## Check sequence of manually edited alignment

In the process of manuall editing a multiple sequence file, it is easy to erroneously delete sequence. This section will demonstrate using `check_seq_in_MSAclustal_consistent_with_FASTA.py` to make sure the sequence in the edited file is valid. It checks against a user-provided FASTA. It is suggested this comes directly from an 'official' source. 

For these proteins, they came from the [Saccharomyces Genome Database (SGD)](https://www.yeastgenome.org/) page for each respective encoding gene, and then from the 'Protein' tab (example: [STV1 protein tab](https://www.yeastgenome.org/locus/S000004658/protein)) under 'Sequence' about half way down the page , I clicked on the button 'Download Sequence (.fsa)'

## Optional: Adjust width of alignment

Biopython produces alignments with 50 sequence characters per line even if you started with ones like those that come from [EMBL-EBI's MUSCLE](https://www.ebi.ac.uk/Tools/msa/muscle/) or similar tools with 60 sequence characters per line. Clustal-formateted data with fifty sequence characters per line can be adjusted back to 60 (or some options) using Mview and then parsing the output to actual CLUSTAL format. (Admittedly, it is a little kludgy but it works and it is less software to maintain by relying on MView for most of the heavy lifting.)

This illustrates doing that. 

## Add a consensus symbol line to an MSA

Multiple sequence alignments from various sources don't come with the consensus symbols line typically provided by [EMBL-EBI's MUSCLE](https://www.ebi.ac.uk/Tools/msa/muscle/). Theese symbols are described [here](https://www.ebi.ac.uk/seqdb/confluence/display/JDSAT/Bioinformatics+Tools+FAQ#BioinformaticsToolsFAQ-WhatdoconsensussymbolsrepresentinaMultipleSequenceAlignment?). Or sometimes they can get lost or need substantial updating following manual editing to the point is easier to remove them and start over to add them correctly. Here `calculate_cons_for_clustal_protein.py` is used to add a consensus line to an multiple sequence alignment.  I have a separate script for nucleic acids, called `calculate_cons_for_clustal_nucleic.py`, see about it [here](https://github.com/fomightez/sequencework/tree/master/alignment-utilities).

Possible subsequent use for the consensus symbols line:  

Beyond visually displaying relatedness in a multiple sequence alignment, these symbols can be used for categorizing residues to make commands for highlighting in molecular visualization. See [here for an example](https://nbviewer.jupyter.org/github/fomightez/cl_demo-binder/blob/master/cl_demo-binder%20Categorize%20conservation%20in%20a%20MSA%20and%20use%20that%20to%20generate%20molvis%20commands.ipynb) that uses `categorize_residues_based_on_conservation_relative_consensus_line.py` script described [here](https://github.com/fomightez/sequencework/tree/master/alignment-utilities). The notebook can be launched in active form from [here](https://github.com/fomightez/cl_demo-binder) and then selecting from the index to go to the 'Categorize conservation in a MSA and use that to generate molvis commands' page. The demo was put in the structure work demo series because it was mainly developed to work towards making commands for molecular visualization.